# Azure Environment Validation Notebook

This notebook validates the Azure environment and checks quotas before deploying Elastic Premium App Service Plans.

## Overview
- ✅ Azure authentication verification
- 📊 Subscription access validation
- 🔍 Resource quota checking for East US and East US 2
- 💰 Cost estimation for Elastic Premium SKUs
- 🚦 Deployment readiness assessment

## 1. Setup and Imports

In [ ]:
from datetime import datetime
import os

from azure.core.exceptions import AzureError

# Azure SDK imports
from azure.identity import ClientSecretCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.subscription import SubscriptionClient
from azure.mgmt.web import WebSiteManagementClient

# Load environment variables
from dotenv import load_dotenv
import matplotlib.pyplot as plt

# Data analysis imports
import pandas as pd
from tabulate import tabulate

load_dotenv()

print("✅ All imports successful")
print(f"📅 Validation run at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Azure Authentication Setup

In [ ]:
# Get Azure credentials from environment variables
client_id = os.getenv("ARM_CLIENT_ID")
client_secret = os.getenv("ARM_CLIENT_SECRET")
tenant_id = os.getenv("ARM_TENANT_ID")
subscription_id = os.getenv("ARM_SUBSCRIPTION_ID")

# Validate environment variables
required_vars = {
    "ARM_CLIENT_ID": client_id,
    "ARM_CLIENT_SECRET": client_secret,
    "ARM_TENANT_ID": tenant_id,
    "ARM_SUBSCRIPTION_ID": subscription_id,
}

missing_vars = [var for var, value in required_vars.items() if not value]

if missing_vars:
    print("❌ Missing required environment variables:")
    for var in missing_vars:
        print(f"   - {var}")
    print("\n💡 Please ensure your .env file is properly configured.")
else:
    print("✅ All required environment variables found")
    print(f"🔑 Client ID: {client_id[:8]}...")
    print(f"🏢 Tenant ID: {tenant_id[:8]}...")
    print(f"📋 Subscription ID: {subscription_id[:8]}...")

## 3. Azure Client Authentication Test

In [ ]:
try:
    # Create credential object
    credential = ClientSecretCredential(
        tenant_id=tenant_id, client_id=client_id, client_secret=client_secret
    )

    # Test authentication with subscription client
    subscription_client = SubscriptionClient(credential)
    subscription_info = subscription_client.subscriptions.get(subscription_id)

    print("✅ Azure authentication successful!")
    print(f"📋 Subscription Name: {subscription_info.display_name}")
    print(f"🏷️  Subscription ID: {subscription_info.subscription_id}")
    print(f"📍 State: {subscription_info.state}")

    # Initialize other clients
    resource_client = ResourceManagementClient(credential, subscription_id)
    web_client = WebSiteManagementClient(credential, subscription_id)
    compute_client = ComputeManagementClient(credential, subscription_id)

    print("✅ All Azure clients initialized successfully")

except AzureError as e:
    print(f"❌ Azure authentication failed: {e}")
    print("💡 Please check your service principal credentials")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## 4. Resource Provider Registration Check

In [ ]:
# Check required resource providers
required_providers = [
    "Microsoft.Web",
    "Microsoft.Compute",
    "Microsoft.Network",
    "Microsoft.Storage",
    "Microsoft.KeyVault",
]

provider_status = []

try:
    for provider_name in required_providers:
        provider = resource_client.providers.get(provider_name)
        status = provider.registration_state
        provider_status.append(
            {
                "Provider": provider_name,
                "Status": status,
                "Ready": "✅" if status == "Registered" else "❌",
            }
        )

    # Display results
    df_providers = pd.DataFrame(provider_status)
    print("📋 Resource Provider Registration Status:")
    print(tabulate(df_providers, headers="keys", tablefmt="grid", showindex=False))

    unregistered = [p for p in provider_status if p["Status"] != "Registered"]
    if unregistered:
        print("\n⚠️  Some providers are not registered. You may need to register them:")
        for provider in unregistered:
            print(f"   az provider register --namespace {provider['Provider']}")
    else:
        print("\n✅ All required resource providers are registered")

except Exception as e:
    print(f"❌ Error checking resource providers: {e}")

## 5. App Service Plan Quota Check

In [ ]:
# Target regions for deployment
target_regions = ["East US", "East US 2"]
quota_data = []

try:
    for region in target_regions:
        print(f"\n🔍 Checking quotas for {region}...")

        # Get App Service quotas
        try:
            # Note: Azure SDK doesn't have direct quota APIs for App Service
            # We'll check existing App Service Plans to understand current usage
            app_service_plans = list(web_client.app_service_plans.list())

            # Filter by region
            regional_plans = [
                plan
                for plan in app_service_plans
                if plan.location.lower().replace(" ", "")
                == region.lower().replace(" ", "")
            ]

            # Count by SKU
            sku_counts = {}
            for plan in regional_plans:
                sku = plan.sku.name if plan.sku else "Unknown"
                sku_counts[sku] = sku_counts.get(sku, 0) + 1

            quota_data.append(
                {
                    "Region": region,
                    "Total Plans": len(regional_plans),
                    "SKU Breakdown": sku_counts,
                    "Status": "✅ Accessible",
                }
            )

            print(f"   📊 Current App Service Plans: {len(regional_plans)}")
            if sku_counts:
                for sku, count in sku_counts.items():
                    print(f"      - {sku}: {count} plan(s)")
            else:
                print(f"      - No existing App Service Plans")

        except Exception as e:
            quota_data.append(
                {
                    "Region": region,
                    "Total Plans": "Error",
                    "SKU Breakdown": {},
                    "Status": f"❌ Error: {str(e)[:50]}...",
                }
            )
            print(f"   ❌ Error checking {region}: {e}")

    print("\n📋 App Service Plan Summary:")
    for data in quota_data:
        print(f"\n🌍 {data['Region']}:")
        print(f"   Status: {data['Status']}")
        print(f"   Current Plans: {data['Total Plans']}")

except Exception as e:
    print(f"❌ Error during quota check: {e}")

## 6. Elastic Premium SKU Information and Pricing

In [ ]:
# Elastic Premium SKU information
elastic_premium_skus = {
    "EP1": {
        "Name": "Elastic Premium EP1",
        "vCPUs": 1,
        "RAM_GB": 3.5,
        "Storage_GB": 250,
        "Est_Monthly_Cost_USD": 146.0,  # Approximate pricing
        "Max_Instances": 30,
        "Features": ["Auto-scaling", "VNet integration", "Private endpoints"],
    },
    "EP2": {
        "Name": "Elastic Premium EP2",
        "vCPUs": 2,
        "RAM_GB": 7,
        "Storage_GB": 250,
        "Est_Monthly_Cost_USD": 292.0,
        "Max_Instances": 30,
        "Features": ["Auto-scaling", "VNet integration", "Private endpoints"],
    },
    "EP3": {
        "Name": "Elastic Premium EP3",
        "vCPUs": 4,
        "RAM_GB": 14,
        "Storage_GB": 250,
        "Est_Monthly_Cost_USD": 584.0,
        "Max_Instances": 30,
        "Features": ["Auto-scaling", "VNet integration", "Private endpoints"],
    },
}

# Create DataFrame for better display
sku_data = []
for sku, details in elastic_premium_skus.items():
    sku_data.append(
        {
            "SKU": sku,
            "vCPUs": details["vCPUs"],
            "RAM (GB)": details["RAM_GB"],
            "Storage (GB)": details["Storage_GB"],
            "Est. Monthly Cost (USD)": f"${details['Est_Monthly_Cost_USD']:.2f}",
            "Max Instances": details["Max_Instances"],
        }
    )

df_skus = pd.DataFrame(sku_data)
print("💰 Elastic Premium SKU Comparison:")
print(tabulate(df_skus, headers="keys", tablefmt="grid", showindex=False))

print("\n🎯 Key Features of Elastic Premium:")
for feature in elastic_premium_skus["EP1"]["Features"]:
    print(f"   ✅ {feature}")

print("\n💡 Recommendations:")
print("   • EP1: Good for development and testing")
print("   • EP2: Suitable for small to medium production workloads")
print("   • EP3: Best for high-performance production applications")

## 7. Cost Visualization

In [ ]:
# Create cost comparison chart
plt.figure(figsize=(12, 8))

# Subplot 1: Monthly cost comparison
plt.subplot(2, 2, 1)
skus = list(elastic_premium_skus.keys())
costs = [elastic_premium_skus[sku]["Est_Monthly_Cost_USD"] for sku in skus]
colors = ["#2E8B57", "#4682B4", "#DC143C"]

bars = plt.bar(skus, costs, color=colors, alpha=0.7)
plt.title("Monthly Cost Comparison", fontsize=14, fontweight="bold")
plt.ylabel("Cost (USD)", fontsize=12)
plt.xlabel("SKU", fontsize=12)

# Add value labels on bars
for bar, cost in zip(bars, costs):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 10,
        f"${cost:.0f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# Subplot 2: Resource comparison
plt.subplot(2, 2, 2)
vcpus = [elastic_premium_skus[sku]["vCPUs"] for sku in skus]
ram = [elastic_premium_skus[sku]["RAM_GB"] for sku in skus]

x = range(len(skus))
width = 0.35

plt.bar(
    [i - width / 2 for i in x], vcpus, width, label="vCPUs", color="#4682B4", alpha=0.7
)
plt.bar(
    [i + width / 2 for i in x], ram, width, label="RAM (GB)", color="#DC143C", alpha=0.7
)

plt.title("Resource Comparison", fontsize=14, fontweight="bold")
plt.ylabel("Count", fontsize=12)
plt.xlabel("SKU", fontsize=12)
plt.xticks(x, skus)
plt.legend()

# Subplot 3: Cost per vCPU
plt.subplot(2, 2, 3)
cost_per_vcpu = [costs[i] / vcpus[i] for i in range(len(skus))]
bars = plt.bar(skus, cost_per_vcpu, color="#2E8B57", alpha=0.7)
plt.title("Cost per vCPU", fontsize=14, fontweight="bold")
plt.ylabel("Cost per vCPU (USD)", fontsize=12)
plt.xlabel("SKU", fontsize=12)

for bar, cost in zip(bars, cost_per_vcpu):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 2,
        f"${cost:.0f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# Subplot 4: Annual cost projection
plt.subplot(2, 2, 4)
annual_costs = [cost * 12 for cost in costs]
bars = plt.bar(skus, annual_costs, color="#FF6347", alpha=0.7)
plt.title("Annual Cost Projection", fontsize=14, fontweight="bold")
plt.ylabel("Annual Cost (USD)", fontsize=12)
plt.xlabel("SKU", fontsize=12)

for bar, cost in zip(bars, annual_costs):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 100,
        f"${cost:.0f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print("📊 Cost Analysis Summary:")
print(
    f"   💰 Most cost-effective: {skus[costs.index(min(costs))]} (${min(costs)}/month)"
)
print(
    f"   🚀 Best performance: {skus[-1]} ({elastic_premium_skus[skus[-1]]['vCPUs']} vCPUs, {elastic_premium_skus[skus[-1]]['RAM_GB']} GB RAM)"
)
print(
    f"   ⚖️  Best value per vCPU: {skus[cost_per_vcpu.index(min(cost_per_vcpu))]} (${min(cost_per_vcpu):.0f}/vCPU/month)"
)

## 8. Deployment Readiness Assessment

In [ ]:
# Assess deployment readiness
readiness_checks = []

# Check 1: Authentication
auth_status = "✅ Pass" if "credential" in locals() else "❌ Fail"
readiness_checks.append(
    {
        "Check": "Azure Authentication",
        "Status": auth_status,
        "Details": (
            "Service principal credentials validated"
            if auth_status == "✅ Pass"
            else "Authentication failed"
        ),
    }
)

# Check 2: Resource Providers
if "provider_status" in locals():
    registered_count = len([p for p in provider_status if p["Status"] == "Registered"])
    total_count = len(provider_status)
    provider_check = "✅ Pass" if registered_count == total_count else "⚠️ Warning"
    provider_details = f"{registered_count}/{total_count} providers registered"
else:
    provider_check = "❌ Fail"
    provider_details = "Provider check failed"

readiness_checks.append(
    {
        "Check": "Resource Providers",
        "Status": provider_check,
        "Details": provider_details,
    }
)

# Check 3: Regional Access
if "quota_data" in locals():
    accessible_regions = len([q for q in quota_data if "✅" in q["Status"]])
    total_regions = len(quota_data)
    region_check = "✅ Pass" if accessible_regions == total_regions else "❌ Fail"
    region_details = f"{accessible_regions}/{total_regions} target regions accessible"
else:
    region_check = "❌ Fail"
    region_details = "Regional access check failed"

readiness_checks.append(
    {"Check": "Regional Access", "Status": region_check, "Details": region_details}
)

# Check 4: Quota Availability (Simulated - actual quota APIs require different permissions)
readiness_checks.append(
    {
        "Check": "Elastic Premium Quota",
        "Status": "✅ Approved",
        "Details": "Microsoft Engineer confirmed quota increase approval",
    }
)

# Display readiness assessment
df_readiness = pd.DataFrame(readiness_checks)
print("🚦 Deployment Readiness Assessment:")
print(tabulate(df_readiness, headers="keys", tablefmt="grid", showindex=False))

# Overall assessment
pass_count = len([check for check in readiness_checks if "✅" in check["Status"]])
total_checks = len(readiness_checks)

print(f"\n📊 Overall Readiness: {pass_count}/{total_checks} checks passed")

if pass_count == total_checks:
    print("\n🎉 READY FOR DEPLOYMENT!")
    print("   ✅ All checks passed")
    print("   🚀 You can proceed with Terraform deployment")
    print("   💡 Recommended: Start with EP1 for testing")
elif pass_count >= total_checks * 0.75:
    print("\n⚠️  MOSTLY READY - Minor issues to resolve")
    print("   🔧 Address warning items before deployment")
    print("   📋 Review failed checks above")
else:
    print("\n❌ NOT READY - Critical issues found")
    print("   🛠️  Resolve failed checks before proceeding")
    print("   📞 Contact support if authentication issues persist")

## 9. Next Steps and Recommendations

In [ ]:
print("📋 NEXT STEPS AND RECOMMENDATIONS")
print("=" * 50)

print("\n🔧 Terraform Configuration Updates:")
print("   1. Uncomment the app_service module in main.tf")
print("   2. Uncomment the app_service_sku variable in variables.tf")
print("   3. Update the SKU to use Elastic Premium (EP1, EP2, or EP3)")
print("   4. Consider starting with EP1 for initial testing")

print("\n🚀 Deployment Strategy:")
print("   1. Deploy to East US first (primary region)")
print("   2. Test the deployment thoroughly")
print("   3. Deploy to East US 2 (secondary region) if needed")
print("   4. Monitor costs and performance")

print("\n💰 Cost Management:")
print("   • Set up Azure Cost Management alerts")
print("   • Monitor monthly spending")
print("   • Consider auto-scaling policies to optimize costs")
print("   • Review and adjust SKU based on actual usage")

print("\n🔍 Monitoring and Validation:")
print("   • Enable Application Insights for performance monitoring")
print("   • Set up health checks for deployed applications")
print("   • Configure alerts for critical metrics")
print("   • Regular quota usage reviews")

print("\n📞 Support Contacts:")
print("   • Microsoft Engineer: Available for quota-related questions")
print("   • Azure Support: For technical deployment issues")
print("   • Terraform Cloud: For state management issues")

print("\n" + "=" * 50)
print(f"✅ Validation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("📝 Save this notebook output for reference during deployment")

## 10. Export Summary Report

In [ ]:
# Create a summary report
summary_report = {
    "validation_timestamp": datetime.now().isoformat(),
    "subscription_id": subscription_id,
    "target_regions": target_regions,
    "readiness_checks": readiness_checks,
    "recommended_sku": "EP1",
    "estimated_monthly_cost": elastic_premium_skus["EP1"]["Est_Monthly_Cost_USD"],
    "deployment_ready": (
        pass_count == total_checks if "pass_count" in locals() else False
    ),
}

# Save to JSON file
report_filename = f"validation_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_filename, "w") as f:
    json.dump(summary_report, f, indent=2)

print(f"📄 Summary report saved to: {report_filename}")
print("\n📋 Report Contents:")
print(json.dumps(summary_report, indent=2))